# Zero to Snowflake - AISQL Functions

## トピック
1. SENTIMENT()を使用してトラックの顧客レビューをスコア化・ラベル付け
2. AI_CLASSIFY()を使用してレビューを分類
3. EXTRACT_ANSWER()を使用してレビューテキストから特定の情報を抽出
4. AI_SUMMARIZE_AGG()を使用して顧客センチメントの要約を生成

## セッション設定

In [ ]:
%%sql
ALTER SESSION SET query_tag = '{"origin":"sf_sit-is","name":"tb_101_v2_aisql","version":{"major":1, "minor":1},"attributes":{"is_quickstart":0, "source":"tastybytes", "vignette": "aisql_functions"}}';

In [ ]:
%%sql
USE ROLE tb_analyst;
USE DATABASE tb_101;
USE WAREHOUSE tb_analyst_wh;

## 1. 大規模なセンチメント分析

SENTIMENT()関数を使用して、顧客レビューを-1（ネガティブ）から+1（ポジティブ）まで自動的にスコア化します。

In [ ]:
%%sql
-- 各トラックブランドの顧客センチメント分析
SELECT
    truck_brand_name,
    COUNT(*) AS total_reviews,
    AVG(CASE WHEN sentiment >= 0.5 THEN sentiment END) AS avg_positive_score,
    AVG(CASE WHEN sentiment BETWEEN -0.5 AND 0.5 THEN sentiment END) AS avg_neutral_score,
    AVG(CASE WHEN sentiment <= -0.5 THEN sentiment END) AS avg_negative_score
FROM (
    SELECT
        truck_brand_name,
        SNOWFLAKE.CORTEX.SENTIMENT (review) AS sentiment
    FROM harmonized.truck_reviews_v
    WHERE
        language ILIKE '%en%'
        AND review IS NOT NULL
    LIMIT 10000
)
GROUP BY
    truck_brand_name
ORDER BY total_reviews DESC;

## 2. 顧客フィードバックの分類

AI_CLASSIFY()関数を使用して、レビューをビジネス関連のカテゴリに自動的に分類します。

In [ ]:
%%sql
-- レビューを食品品質、価格、サービス体験、スタッフ行動に分類
WITH classified_reviews AS (
  SELECT
    truck_brand_name,
    AI_CLASSIFY(
      review,
      ['Food Quality', 'Pricing', 'Service Experience', 'Staff Behavior']
    ):labels[0] AS feedback_category
  FROM
    harmonized.truck_reviews_v
  WHERE
    language ILIKE '%en%'
    AND review IS NOT NULL
    AND LENGTH(review) > 30
  LIMIT
    10000
)
SELECT
  truck_brand_name,
  feedback_category,
  COUNT(*) AS number_of_reviews
FROM
  classified_reviews
GROUP BY
  truck_brand_name,
  feedback_category
ORDER BY
  truck_brand_name,
  number_of_reviews DESC;

## 3. 特定の運営インサイトの抽出

EXTRACT_ANSWER()関数を使用して、非構造化テキストから正確な回答を得ます。

In [ ]:
%%sql
-- 各レビュー内で言及されている特定の改善点や苦情を抽出
SELECT
    truck_brand_name,
    primary_city,
    LEFT(review, 100) || '...' AS review_preview,
    SNOWFLAKE.CORTEX.EXTRACT_ANSWER(
        review,
        'What specific improvement or complaint is mentioned in this review?'
    ) AS specific_feedback
FROM 
    harmonized.truck_reviews_v
WHERE 
    language = 'en'
    AND review IS NOT NULL
    AND LENGTH(review) > 50
ORDER BY truck_brand_name, primary_city ASC
LIMIT 10000;

## 4. エグゼクティブサマリーの生成

AI_SUMMARIZE_AGG()関数を使用して、顧客フィードバックの簡潔な要約を作成します。

In [ ]:
%%sql
-- 各トラックブランドのレビュー要約を生成
SELECT
  truck_brand_name,
  AI_SUMMARIZE_AGG (review) AS review_summary
FROM
  (
    SELECT
      truck_brand_name,
      review
    FROM
      harmonized.truck_reviews_v
    LIMIT
      100
  )
GROUP BY
  truck_brand_name;